In [34]:
from dimod import (
    Binary,
    BinaryQuadraticModel,
    ConstrainedQuadraticModel,
)

from dwave.system import LeapHybridCQMSampler

import random
import pandas as pd
import numpy as np

# Define Token here to run
from credentials import TOKEN

from process_output import get_routes_from_sample, get_cost_routes, report_output, check_feasibility_sample

from vrp import VehicleRoutingProblem

ImportError: cannot import name 'check_feasibility_sample' from 'process_output' (c:\Users\brenn\OneDrive - Yale University\Desktop\NYUAD\new_vrp\Vehicle Routing\process_output.py)

In [8]:
def build_vrp_cqm(num_locations, distances, num_vehicles, max_distance):
    """
    Input:
        num_locations: number of distinct locations for orders
        distances: an matrix with the distances between respective locations for a fully connected graph
        num_vehicles: the number of vehicles used
        max_distance: the maximum distance that a truck can drive in one day
    Output:
        A constrained binary quadratic model representing a CVRP
    """

    M = num_vehicles
    N = num_locations

    cqm = ConstrainedQuadraticModel()

    # Create the binary variables and objective
    x, obj = VehicleRoutingProblem.construct_objective(num_locations, distances, num_vehicles)

    cqm.set_objective(obj)

    # Implement constraints:

    # 1. Each location should be served by exactly one vehicle (only checks first N because depot is required start and end location by construction)
    for j in range(N):
        sum = 0
        for m in range(M):
            for k in range(N):
                sum += x[m, j, k]

        cqm.add_constraint(sum == 1,
                           label=f"Vertex {j} is not visited or visited more than once")

    # 2. Each vehicle is in one location
    for i in range(M):
        for k in range(N):
            sum = 0
            for j in range(N + 1):
                sum += x[i, j, k]
            cqm.add_constraint(sum == 1,
                               label=f"Vehicle {i} is at more or less than one position at time {k}")
            
    #3. Each vehicle drives less than the cap
    for m in range(M):
        sum = 0

        for i in range(N+1):
            
            # Add in the distances from deport to first, last to depot
            sum += x[m, i, 0]*distances[N][i]
            sum += x[m, i, N-1]*distances[i][N]

            # Go through the steps in the middle
            for j in range(N+1):
                for k in range(N-1):
                    sum += x[m, i, k]*x[m, j, k+1]*distances[i][j]

        cqm.add_constraint(sum <= max_distance,
                            label=f"Vehicle {m} drives more than the maximum capacity")

            
    # Return the constrained model
    return cqm

In [9]:
def run_cqm(cqm):
    """Run the provided CQM on the Leap Hybrid CQM Sampler with postselection of feasible solutions"""
    sampler = LeapHybridCQMSampler(token=TOKEN)

    sampleset = sampler.sample_cqm(cqm)
    feasible_sampleset = sampleset.filter(lambda row: row.is_feasible)

    num_feasible = len(feasible_sampleset)
    errors = " "
    if num_feasible == 0:
        print("No feasible solution found.")
        return sampleset

    print("\nFeasible solution found.\n")

    return feasible_sampleset

In [10]:
# Create a sample problem
num_destinations = 5
num_vehicles = 1
max_distance = 300

# Generate a random symmetric cost matrix
cost_matrix = [[0]*(num_destinations+1) for _ in range(num_destinations + 1)]

for i in range(num_destinations + 1):
    for j in range(i, num_destinations + 1):
        # Select random values that do not violate the triangle inequality
        if i == j:
            cost_matrix[i][j] = 0
        else:
            val = random.randint(5,9)
            cost_matrix[i][j] = val
            cost_matrix[j][i] = val

# Print the adjacency matrix
for row in cost_matrix:
    print(row)

[0, 8, 6, 7, 9, 8]
[8, 0, 9, 5, 7, 9]
[6, 9, 0, 5, 9, 5]
[7, 5, 5, 0, 6, 9]
[9, 7, 9, 6, 0, 8]
[8, 9, 5, 9, 8, 0]


### Test 1: Traveling Salesman Problem

In [11]:
cqm = build_vrp_cqm(num_destinations, cost_matrix, num_vehicles, max_distance)

In [12]:
feasible_sampleset = run_cqm(cqm)


Feasible solution found.



In [13]:
lowest_energy_sample = feasible_sampleset.lowest().first.sample

routes = get_routes_from_sample(lowest_energy_sample, num_vehicles, num_destinations)

print(f'Best route: {routes}')

print(f'Best cost: {feasible_sampleset.lowest().first.energy}')

print(f'Feasible: {check_feasibility_sample(lowest_energy_sample, num_vehicles, num_destinations, cost_matrix, max_distance)}')


#print(get_cost_routes(routes, cost_matrix))

# print(feasible_sampleset)

Best route: [[4, 3, 1, 0, 2]]
Best cost: 38.0


In [32]:
# Lazy sanity check
r = []
for i in range(5):
    r.append(i)

min = 300
best_route = r
# Randomly sample a lot of options
for i in range(10000):
    random.shuffle(r)

    cost = get_cost_routes([r], cost_matrix)
    if cost < min:
        min = cost
        best_routes = [r]
print(min)
print(best_routes)

38
[[3, 2, 0, 4, 1]]


### Test 2: Capacitated Multiple Vehicle Routing Problem

In [14]:
cqm2 = build_vrp_cqm(num_destinations, cost_matrix, num_vehicles=2, max_distance=30)

In [15]:
feasible_sampleset2 = run_cqm(cqm2)


Feasible solution found.



In [16]:
lowest_energy_sample2 = feasible_sampleset2.lowest().first.sample

routes2 = get_routes_from_sample(lowest_energy_sample2, num_vehicles=2, num_steps=num_destinations)

report_output(routes2, cost_matrix)

Best routes (depot omitted at start and end):
	Vehicle 0: [0]
	Vehicle 1: [4, 1, 3, 2]
Best cost: 46


In [31]:
# Lazy sanity check
r = []
for i in range(5):
    r.append(i)

min = 300
best_route = r
# Randomly sample a lot of options
for i in range(10000):
    random.shuffle(r)

    r1 = r[:2]
    r2 = r[2:]

    cost = get_cost_routes([r1, r2], cost_matrix)
    if cost < min:
        min = cost
        best_routes = [r1, r2]
print(min)
print(best_routes)

47
[[0, 2], [4, 3, 1]]
